## Chapter 8: `MultisetEvaluator`s

By implementing your own `MultisetEvaluator`, you can create your own efficient pool evaluation.

### Transition function (`next_state`)

The core of an `MultisetEvaluator` is the transition function `next_state`. Imagine this procedure for evaluating the roll of a pool of d6s:

* First, I tell you how many ones you rolled.
* Then, I tell you how many twos you rolled.
* Then, I tell you how many threes you rolled.
* Then, I tell you how many fours you rolled.
* Then, I tell you how many fives you rolled.
* Then, I tell you how many sixes you rolled.

You're allowed to remember things between each step, typically a "running total" of some sort, ideally without too many distinct possibilities.

Or in terms of Python code:

```python
state = None
state = evaluator.next_state(state, 1, num_ones)
state = evaluator.next_state(state, 2, num_twos)
state = evaluator.next_state(state, 3, num_threes)
state = evaluator.next_state(state, 4, num_fours)
state = evaluator.next_state(state, 5, num_fives)
state = evaluator.next_state(state, 6, num_sixes)
```

The `state` can be anything you want, as long as it's hashable.

The built-in pool evaluation methods are implemented this way. For example, the basic case of `pool.sum()` is implemented as

In [1]:
import piplite
await piplite.install("icepool")

from icepool import MultisetEvaluator, d6

class SumEvaluator(MultisetEvaluator):
    def next_state(self, state, outcome, count):
        """Add the outcomes to the running total. """
        if state is None:
            return outcome * count
        else:
            return state + outcome * count

# Example of running an evaluator.
evaluator = SumEvaluator()
print(evaluator(d6.pool(3)))

Die with denominator 216

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       3 |        1 |   0.462963% |
|       4 |        3 |   1.388889% |
|       5 |        6 |   2.777778% |
|       6 |       10 |   4.629630% |
|       7 |       15 |   6.944444% |
|       8 |       21 |   9.722222% |
|       9 |       25 |  11.574074% |
|      10 |       27 |  12.500000% |
|      11 |       27 |  12.500000% |
|      12 |       25 |  11.574074% |
|      13 |       21 |   9.722222% |
|      14 |       15 |   6.944444% |
|      15 |       10 |   4.629630% |
|      16 |        6 |   2.777778% |
|      17 |        3 |   1.388889% |
|      18 |        1 |   0.462963% |




Here's another example: finding the size of the largest matching set.

In [2]:
class LargestMatchingSetEvaluator(MultisetEvaluator):
    def next_state(self, state, outcome, count):
        if state is None:
            return count
        else:
            return max(state, count)

largest_matching_set_evaluator = LargestMatchingSetEvaluator()
print(largest_matching_set_evaluator(d6.pool(5)))

Die with denominator 7776

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |      720 |   9.259259% |
|       2 |     5400 |  69.444444% |
|       3 |     1500 |  19.290123% |
|       4 |      150 |   1.929012% |
|       5 |        6 |   0.077160% |




### `final_outcome`

`next_state` is the only strictly-required abstract method to implement. However, there are some optional methods that you may find useful to implement. 

`final_outcome` allows you to fix up the final state before producing the result. This is useful if there is some final calculation you want to do, or if you simply want to drop some information that you no longer care about. For example, here's an evaluator that finds the length of the longest straight:

In [3]:
class LongestStraightEvaluator(MultisetEvaluator):

    def next_state(self, state, outcome, count):
        """Increments the current run if at least one `Die` rolled this outcome,
        then saves the run to the state.
        """
        best_run, run, prev_outcome = state or (0, 0, outcome - 1)
        if outcome == prev_outcome + 1 and count >= 1:
            run += 1
        else:
            run = 0
        return max(best_run, run), run, outcome

    def final_outcome(self, final_state):
        # Return just the length of the best run.
        return final_state[0]

longest_straight_evaluator = LongestStraightEvaluator()
print(longest_straight_evaluator(d6.pool(5)))

Die with denominator 7776

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |      906 |  11.651235% |
|       2 |     3390 |  43.595679% |
|       3 |     2280 |  29.320988% |
|       4 |      960 |  12.345679% |
|       5 |      240 |   3.086420% |




### `alignment`

`alignment` allows you to guarantee that certain outcomes are not skipped even if they have zero count. For example, in the case of straights, this allows us to skip checking the previous outcome.

In [4]:
class LongestStraightEvaluator(MultisetEvaluator):

    def next_state(self, state, outcome, count):
        best_run, run = state or (0, 0)
        if count >= 1:
            run += 1
        else:
            run = 0
        return max(best_run, run), run

    def final_outcome(self, final_state):
        # Return just the length of the best run.
        return final_state[0]
    
    # This guarantees that we see all consecutive integers.
    alignment = MultisetEvaluator.range_alignment
    
longest_straight_evaluator = LongestStraightEvaluator()
print(longest_straight_evaluator(d6.pool(5)))

Die with denominator 7776

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |      906 |  11.651235% |
|       2 |     3390 |  43.595679% |
|       3 |     2280 |  29.320988% |
|       4 |      960 |  12.345679% |
|       5 |      240 |   3.086420% |




This is useful when performing an evaluation on a pool consisting of fixed values, ensuring that any gaps are not skipped:

In [5]:
class BadStraightEvaluator(MultisetEvaluator):

    def next_state(self, state, outcome, count):
        best_run, run = state or (0, 0)
        if count >= 1:
            run += 1
        else:
            run = 0
        return max(best_run, run), run

    def final_outcome(self, final_state):
        # Return just the length of the best run.
        return final_state[0]
    
    # Missing alignment.

bad_evaluator = BadStraightEvaluator()
print(bad_evaluator([1, 2, 2, 5, 6]))
print(longest_straight_evaluator([1, 2, 2, 5, 6]))

Die with denominator 1

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       4 |        1 | 100.000000% |


Die with denominator 1

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       2 |        1 | 100.000000% |




### `order`

`order` allows you to specify whether the outcomes are seen in ascending order (default), descending order, or to declare that the evaluator does not care. For example, if we also wanted to produce the greatest outcome in the best straight:

In [6]:
from icepool import Order

class BestStraightEvaluator(MultisetEvaluator):

    def next_state(self, state, outcome, count):
        best_run, run = state or ((0, outcome), 0)
        if count >= 1:
            run += 1
        else:
            run = 0
        return max(best_run, (run, outcome)), run

    def final_outcome(self, final_state):
        return final_state[0]
    
    def order(self, *_):
        return Order.Ascending
    
    # This guarantees that we see all consecutive integers.
    alignment = MultisetEvaluator.range_alignment

best_straight_evaluator = BestStraightEvaluator()
print(best_straight_evaluator(d6.pool(5)))

Die with denominator 7776

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          1 |          1 |        1 |   0.012860% |
|          1 |          2 |        1 |   0.012860% |
|          1 |          3 |       31 |   0.398663% |
|          1 |          4 |       61 |   0.784465% |
|          1 |          5 |      241 |   3.099280% |
|          1 |          6 |      571 |   7.343107% |
|          2 |          2 |      720 |   9.259259% |
|          2 |          3 |      330 |   4.243827% |
|          2 |          4 |      570 |   7.330247% |
|          2 |          5 |      570 |   7.330247% |
|          2 |          6 |     1200 |  15.432099% |
|          3 |          3 |      750 |   9.645062% |
|          3 |          4 |      390 |   5.015432% |
|          3 |          5 |      390 |   5.015432% |
|          3 |          6 |      750 |   9.645062% |
|          4 |          4 |      360 |   4.629630% |
|          4 |     

### Multiple pools

You can create an evaluator that looks at multiple pools at a time. In this case, `next_state` simply gets one count per pool. For example, this evaluates a *RISK*-like mechanic:

* Both sides roll a pool of d6s.
* Sort the dice of each side in descending order.
* Pair up the dice, one from each side. Any unpaired dice are discarded.
* Each pair is won by the side that rolled higher.

In [7]:
class RiskEvaluator(MultisetEvaluator):
    def next_state(self, state, outcome, a, b):
        if state is None:
            score_a, score_b, advantage = 0, 0, 0
        else:
            score_a, score_b, advantage = state
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        # We pair them off with newly-rolled dice of the disadvantaged side.
        if advantage > 0:
            score_a += min(b, advantage)
        elif advantage < 0:
            score_b += min(a, -advantage)
        advantage += a - b
        return score_a, score_b, advantage
    
    def final_outcome(self, final_state):
        # Take only the scores.
        return final_state[:2]
    
    def order(self, *_):
        # See outcomes in descending order.
        return Order.Descending

risk_evaluator = RiskEvaluator()

# 3 vs. 2 dice.
print(risk_evaluator(d6.pool(3), d6.pool(2)))

Die with denominator 7776

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          0 |          0 |      381 |   4.899691% |
|          0 |          1 |      915 |  11.766975% |
|          0 |          2 |      979 |  12.590021% |
|          1 |          0 |     1545 |  19.868827% |
|          1 |          1 |     1066 |  13.708848% |
|          2 |          0 |     2890 |  37.165638% |




### Further reading

While you could get the same results using `Pool.expand().map()` to expand all sorted rolls and then evaluate them, `MultisetEvaluator` is typically much more efficient due to a dynamic programming algorithm. If you would like to know more, you can read [my paper on the subject](https://github.com/HighDiceRoller/icepool/blob/main/papers/icepool_preprint.pdf).

```bibtex
    title={Icepool: Efficient Computation of Dice Pool Probabilities},
    author={Albert Julius Liu},
    booktitle={Eighteenth AAAI Conference on Artificial Intelligence and Interactive Digital Entertainment},
    volume={18},
    number={1},
    pages={258-265},
    year={2022},
    month={Oct.},
    eventdate={2022-10-24/2022-10-28},
    venue={Pomona, California},
    url={https://ojs.aaai.org/index.php/AIIDE/article/view/21971},
    doi={10.1609/aiide.v18i1.21971}
```